In [2]:
# -*- coding: utf-8 -*-
"""Russia_SentimentAnalysis.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1U2rqhaoDv9j4uMRppLST4yuB0DGiUYL7
"""

# prompt: texte recognition (sentiment analysis) for russia language, save the model to test it after

!pip install transformers
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# Load the pre-trained model and tokenizer for Russian sentiment analysis
model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Example usage
text = "Это очень хороший фильм."  # "This is a very good movie." in Russian
result = sentiment_pipeline(text)
print(result)


# Save the model and tokenizer for later use
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")


# Load the saved model and tokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained("sentiment_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("sentiment_model")

# Create a new pipeline using the loaded model and tokenizer
loaded_sentiment_pipeline = pipeline("sentiment-analysis", model=loaded_model, tokenizer=loaded_tokenizer)

# Test the loaded model
text = "Мне очень нравится эта книга."  # "I really like this book." in Russian
result = loaded_sentiment_pipeline(text)
print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9952924251556396}]
[{'label': 'POSITIVE', 'score': 0.9955089092254639}]


In [ ]:
import random
import time
import threading
import multiprocessing

In [36]:
negative_dataset = []
positive_dateset = []
neutral_dataset = []
dataset = []

with open("positive.txt") as f:
  positive_dateset += [l.split(';') for l in f]

with open("negative.txt") as f:
  negative_dataset += [l.split(';') for l in f]

with open("neutral.txt") as f:
  neutral_dataset += [l.split(';') for l in f]

dataset = positive_dateset + negative_dataset + neutral_dataset
random.shuffle(dataset)

In [10]:
print(dataset)

[['Я взволнован, чтобы начать свое новое приключение.', ' Positive\n'],
 ['Я расстроен постоянными задержками.', ' Negative\n'],
 ['Эта книга была полным разочарованием.', ' Negative\n'],
 ['Цветы в саду потрясающие.', ' Positive\n'],
 ['Я чувствую благодарность за всю поддержку, которую получил.',
  ' Positive\n'],
 ['Я сожалею о покупке этого продукта.', ' Negative\n'],
 ['Обслуживание было фантастическим и превзошло мои ожидания.', ' Positive\n'],
 ['Я чувствую, что мои проблемы игнорируют.', ' Negative\n'],
 ['Мероприятие не оправдало ожиданий.', ' Negative\n'],
 ['Я был недоволен тем, как меня обслужили.', ' Negative\n'],
 ['Персонал был дружелюбным и полезным.', ' Positive\n'],
 ['Я с нетерпением жду предстоящего события.', ' Positive\n'],
 ['Персонал был грубым и пренебрежительным.', ' Negative\n'],
 ['Онлайн-курс был очень полезным.', ' Positive\n'],
 ['Я не был удовлетворен своей едой.', ' Negative\n'],
 ['У меня был негативный опыт с онлайн-заказом.', ' Negative\n'],
 ['Я чув

**Serial sentiment analysis**

In [34]:
start = time.time()
results = 0
for i in dataset:
   result = sentiment_pipeline(i[0])
   #print(result[0]['label'].lower(), i[1].lower().strip('\n'))
  results += int(result[0]['label'].lower().strip() == i[1].lower().strip('\n '))
print("True predict is", results, "in", len(dataset), "dataset.", results / len(dataset) * 100, "%")
end = time.time()
print("Time execution:", end - start)

True predict is 202 in 241 dataset. 0.8381742738589212 %
Time execution: 21.451128721237183


**Multithreading sentiment analysis**

In [45]:
parallel_results = 0
lock = threading.Lock()

def predict(dataset):
  global parallel_results
  for i in dataset:
    result = sentiment_pipeline(i[0])
    if result[0]['label'].lower().strip() == i[1].lower().strip('\n '):
      with lock:
        parallel_results += 1

start = time.time()
t1 = threading.Thread(target=predict, args=(positive_dateset,))
t2 = threading.Thread(target=predict, args=(negative_dataset,))
t3 = threading.Thread(target=predict, args=(neutral_dataset,))

t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()


end = time.time()
print("True predict is", results, "in", len(dataset), "dataset.", results / len(dataset) * 100, "%")
print("Time execution:", end - start)

True predict is 202 in 241 dataset. 83.81742738589212 %
Time execution: 18.706050395965576


**Multiprocessing sentiment analysis**

In [53]:
def multiprocessing_predict(dataset, q):
  results = 0
  for i in dataset:
    result = sentiment_pipeline(i[0])
    results += int(result[0]['label'].lower().strip() == i[1].lower().strip('\n '))
  q.put(results)

start = time.time()

q = multiprocessing.Queue()
p1 = multiprocessing.Process(target=multiprocessing_predict, args=(positive_dateset, q))
p2 = multiprocessing.Process(target=multiprocessing_predict, args=(negative_dataset, q))
p3 = multiprocessing.Process(target=multiprocessing_predict, args=(neutral_dataset, q))

p1.start()
p2.start()
p3.start()
p1.join()
p2.join()
p3.join()

results = 0
while not q.empty():
  results += q.get()

end = time.time()

print("True predict is", results, "in", len(dataset), "dataset.", results / len(dataset) * 100, "%")
print("Time execution:", end - start)

True predict is 202 in 241 dataset. 83.81742738589212 %
Time execution: 26.33144521713257
